# DAT210x - Programming with Python for DS

## Module6- Lab3

This code is intentionally missing! Read the directions on the course lab page!

Growing up, everyone has a hero. For many people, that hero was Muhammad Ali. He taught people it was okay to be proud of who they were, at a time when others would not accept that. He showed people how to stand up for their beliefs in the face of oppression and tyranny. He made people value themselves, and encouraged them care for those around them. He showed us what bravery truly meant, how to be a heck of a boxer, and so much more. Every single person who met Muhammad Ali, either in the ring or outside of it, had a motivating story to share about their encounter.

On June 3, 2016, Muhammad Ali passed away at the age of 74 due to septic shock. Thirty years earlier, he was diagnosed with Parkinson's syndrome, a neurodegenerative condition that doctors attributed to his boxing-related brain injuries.



Parkinson's disease itself is a long-term disorder of the nervous system that affects many aspects of a person's mobility over time. It's characterized by shaking, slowed movement, rigidity, dementia, and depression. In 2013, some 53 million people were diagnosed with it, mostly men. Other famous personalities affected by it include actor Michael J. Fox, and olympic cyclist Davis Phinney.

In this lab, you will be applying SVC to the Parkinson's Data Set, provided courtesy of UCI's Machine Learning Repository. The dataset was created at the University of Oxford, in collaboration with 10 medical centers around the US, along with Intel who developed the device used to record the primary features of the dataset: speech signals. Your goals for this assignment are first to see if it's possible to differentiate between people who have Parkinson's and who don't using SciKit-Learn's support vector classifier, and then to take a first-stab at a naive way of fine-tuning your parameters in an attempt to maximize the accuracy of your testing set.

"I've never really resented hard work because I've always liked it. Up every morning for roadwork. Going to the gymnasium every day at 12 o'clock. I never change my pattern."

In honor of Muhammad Ali and hard work, there is no starter code for this lab. Just follow the instructions below.

In [8]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import time
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [4]:
C = 1
kernel = 'linear'

In [64]:
df = pd.read_csv(r'Datasets\parkinsons.data')
df.head(3)

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 24 columns):
name                195 non-null object
MDVP:Fo(Hz)         195 non-null float64
MDVP:Fhi(Hz)        195 non-null float64
MDVP:Flo(Hz)        195 non-null float64
MDVP:Jitter(%)      195 non-null float64
MDVP:Jitter(Abs)    195 non-null float64
MDVP:RAP            195 non-null float64
MDVP:PPQ            195 non-null float64
Jitter:DDP          195 non-null float64
MDVP:Shimmer        195 non-null float64
MDVP:Shimmer(dB)    195 non-null float64
Shimmer:APQ3        195 non-null float64
Shimmer:APQ5        195 non-null float64
MDVP:APQ            195 non-null float64
Shimmer:DDA         195 non-null float64
NHR                 195 non-null float64
HNR                 195 non-null float64
status              195 non-null int64
RPDE                195 non-null float64
DFA                 195 non-null float64
spread1             195 non-null float64
spread2             195 non-null floa

In [66]:
from sklearn.preprocessing import Normalizer, MaxAbsScaler, MinMaxScaler, KernelCenterer, StandardScaler


In [70]:
X = df.drop(labels=['name', 'status'], axis=1)
y = df['status']
X = StandardScaler().fit_transform(X)

In [72]:
X[:2]

array([[-0.82929965, -0.43616456, -0.95203729,  0.33491386,  0.74975946,
         0.13296255,  0.76080023,  0.13175532,  0.74598495,  0.73953578,
         0.60785907,  1.11914677,  0.33298451,  0.60753214, -0.06789281,
        -0.19322537, -0.8078385 ,  1.76081412,  0.80132308,  0.48047686,
        -0.21053082,  0.86888575],
       [-0.77097169, -0.53097409, -0.05772056,  0.71541825,  1.03767418,
         0.45389169,  1.27680862,  0.45268371,  1.68173116,  1.7684643 ,
         1.54791153,  2.27650439,  1.15945391,  1.54825419, -0.13784252,
        -0.63450828, -0.38752443,  1.83756192,  1.47985265,  1.31118546,
         0.27507712,  1.80360503]])

In [77]:
from sklearn.decomposition import PCA

max_score = 0
for n_components in range(4,15):
    pca = PCA(n_components=n_components)
    X2 = pca.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3, random_state=7)
    model = SVC(C=1.5000000000000002, gamma=0.1)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    if score > max_score:
        max_score = score
        print(max_score)
    

0.8813559322033898
0.9152542372881356
0.9322033898305084


In [80]:
from sklearn.manifold import Isomap

max_score = 0
for n_neighbors in range(2,6):
    for n_components in range(4,7):
        isomap = Isomap(n_neighbors=n_neighbors, n_components=n_components)
        X2 = isomap.fit_transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3, random_state=7)
        model = SVC(C=1.5000000000000002, gamma=0.1)
        model.fit(X_train, y_train)
        if score > max_score:
            max_score = score
            print(max_score)


0.9322033898305084


In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)
model = SVC(C=1.5000000000000002, gamma=0.1)
model.fit(X_train, y_train)
score = model.score(X_test, y_test)

In [57]:
# X_train = Normalizer().fit_transform(X_train)
# X_test = Normalizer().fit_transform(X_test)
# X_train = MaxAbsScaler().fit_transform(X_train)
# X_test = MaxAbsScaler().fit_transform(X_test)
# X_train = MinMaxScaler().fit_transform(X_train)
# X_test = MinMaxScaler().fit_transform(X_test)
# X_train = KernelCenterer().fit_transform(X_train)
# X_test = KernelCenterer().fit_transform(X_test)
# X_train = StandardScaler().fit_transform(X_train)
# X_test = StandardScaler().fit_transform(X_test)

In [58]:
# C = 1.5000000000000002, gamma = 0.1
best_score = 0
for C in np.arange(0.05, 2.00001, 0.05):
    for gamma in np.arange(0.001, 0.100001, 0.001):
        model = SVC(C=C, gamma=gamma)
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        if score > best_score:
            best_score = score
            print('C = {}, gamma = {}, score = {}'.format(C, gamma, best_score))


C = 0.05, gamma = 0.001, score = 0.7966101694915254
C = 0.15000000000000002, gamma = 0.030000000000000002, score = 0.8305084745762712
C = 0.2, gamma = 0.024, score = 0.864406779661017
C = 0.2, gamma = 0.042, score = 0.8813559322033898
C = 0.45, gamma = 0.030000000000000002, score = 0.8983050847457628
C = 0.7500000000000001, gamma = 0.037000000000000005, score = 0.9152542372881356
C = 1.5000000000000002, gamma = 0.1, score = 0.9322033898305084
